# text preprocessing

In [95]:
# Import libraries
import pandas as pd
import seaborn as sns
import numpy as np
import re
import nltk
import warnings
import random
import string
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from datetime import date
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn import preprocessing 
from scipy import stats
from scipy.stats import norm, skewnorm
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
pd.set_option('display.max_columns', 100)
from nltk import sent_tokenize, word_tokenize
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [96]:
#Load data
df_customers = pd.read_csv('C:\Data\olist_customers_dataset.csv', delimiter = ',',quotechar="'")
df_order_reviews = pd.read_csv('C:\Data\olist_order_reviews_dataset.csv', delimiter = ',', encoding = 'utf-8')
df_geolocations = pd.read_csv('C:\Data\olist_geolocation_dataset.csv', delimiter = ',' )
df_order_payments = pd.read_csv('C:\Data\olist_order_payments_dataset.csv', delimiter = ',', quotechar="'")
df_categorys = pd.read_csv('C:\Data\product_category_name_translation.csv', sep = ',')
df_sellers = pd.read_csv('C:\Data\olist_sellers_dataset.csv', delimiter = ',')
df_products = pd.read_csv('C:\Data\olist_products_dataset.csv', delimiter = ',', quotechar="'")
df_orders_items = pd.read_csv('C:\Data\olist_order_items_dataset.csv', delimiter = ',')
df_orders = pd.read_csv('C:\Data\olist_orders_dataset.csv', delimiter = ',', quotechar="'", encoding = 'utf-8')

##### text variable


In [132]:
text_copy = main_df.copy()

In [133]:
text_copy.isnull().sum()

category                    1709
score                          3
message                    67653
order_record               73759
order_approved             73763
estimated_delivery_date    73759
delivery_date              74754
payment_value                  3
payment_type                   3
shipping_limit_date            0
seller_state                   0
dtype: int64

In [134]:
text_copy.drop_duplicates(keep='first', inplace=True) 

In [135]:
text_copy.head()

,category,score,message,order_record,order_approved,estimated_delivery_date,delivery_date,payment_value,payment_type,shipping_limit_date,seller_state
0,cool_stuff,5.0,"Perfeito, produto entregue antes do combinado.",NaN,NaN,NaN,NaN,72.19,credit_card,2017-09-19 09:45:35,SP
1,pet_shop,4.0,NaN,NaN,NaN,NaN,NaN,259.83,credit_card,2017-05-03 11:05:13,SP
2,moveis_decoracao,5.0,Chegou antes do prazo previsto e o produto surpreendeu pela qualidade. Muito satisfatório.,NaN,NaN,NaN,NaN,216.87,credit_card,2018-01-18 14:48:30,MG
3,perfumaria,4.0,NaN,NaN,NaN,NaN,NaN,25.78,credit_card,2018-08-15 10:10:18,SP
4,ferramentas_jardim,5.0,Gostei pois veio no prazo determinado .,NaN,NaN,NaN,NaN,218.04,credit_card,2017-02-13 13:57:51,PR


In [136]:
text_copy['Tag'] = text_copy['message'].isnull()*1
text_copy

,category,score,message,order_record,order_approved,estimated_delivery_date,delivery_date,payment_value,payment_type,shipping_limit_date,seller_state,Tag
0,cool_stuff,5.0,"Perfeito, produto entregue antes do combinado.",NaN,NaN,NaN,NaN,72.19,credit_card,2017-09-19 09:45:35,SP,0
1,pet_shop,4.0,NaN,NaN,NaN,NaN,NaN,259.83,credit_card,2017-05-03 11:05:13,SP,1
2,moveis_decoracao,5.0,Chegou antes do prazo previsto e o produto surpreendeu pela qualidade. Muito satisfatório.,NaN,NaN,NaN,NaN,216.87,credit_card,2018-01-18 14:48:30,MG,0
3,perfumaria,4.0,NaN,NaN,NaN,NaN,NaN,25.78,credit_card,2018-08-15 10:10:18,SP,1
4,ferramentas_jardim,5.0,Gostei pois veio no prazo determinado .,NaN,NaN,NaN,NaN,218.04,credit_card,2017-02-13 13:57:51,PR,0
...,...,...,...,...,...,...,...,...,...,...,...,...
118313,utilidades_domesticas,5.0,NaN,2018-04-23 13:57:06,2018-04-25 04:11:01,2018-05-18 00:00:00,2018-05-10 22:56:40,343.40,boleto,2018-05-02 04:11:01,SC,1
118314,informatica_acessorios,5.0,NaN,2018-07-14 10:26:46,2018-07-17 04:31:48,2018-08-01 00:00:00,2018-07-23 20:31:55,386.53,boleto,2018-07-20 04:31:48,SP,1
118315,esporte_lazer,5.0,NaN,2017-10-23 17:07:56,2017-10-24 17:14:25,2017-11-10 00:00:00,2017-10-28 12:22:22,116.85,credit_card,2017-10-30 17:14:25,PR,1
118316,informatica_acessorios,5.0,Recebi o produto antes do prazo e corretamente.,2017-08-14 23:02:59,2017-08-15 00:04:32,2017-08-25 00:00:00,2017-08-16 21:59:40,64.71,credit_card,2017-08-21 00:04:32,SP,0


In [137]:
text_copy = text_copy[pd.notnull(text_copy['message'])]
text_copy

,category,score,message,order_record,order_approved,estimated_delivery_date,delivery_date,payment_value,payment_type,shipping_limit_date,seller_state,Tag
0,cool_stuff,5.0,"Perfeito, produto entregue antes do combinado.",NaN,NaN,NaN,NaN,72.19,credit_card,2017-09-19 09:45:35,SP,0
2,moveis_decoracao,5.0,Chegou antes do prazo previsto e o produto surpreendeu pela qualidade. Muito satisfatório.,NaN,NaN,NaN,NaN,216.87,credit_card,2018-01-18 14:48:30,MG,0
4,ferramentas_jardim,5.0,Gostei pois veio no prazo determinado .,NaN,NaN,NaN,NaN,218.04,credit_card,2017-02-13 13:57:51,PR,0
8,beleza_saude,1.0,"Na descrição do produto quando fui efetuar a compra, estava detalhado os pentes de disfarce.\r\nQuando recebe o produto verifiquei e notei que não havia os pentes 0.5 e 1.5",NaN,NaN,NaN,NaN,157.60,credit_card,2018-03-26 18:31:29,SP,0
10,beleza_saude,5.0,"A caixa do produto veio com uma pequena avariação, mas de resto veio perfeito...",NaN,NaN,NaN,NaN,68.87,credit_card,2018-03-29 22:28:09,SP,0
...,...,...,...,...,...,...,...,...,...,...,...,...
118282,beleza_saude,4.0,"Amo comprar no baratheon, sempre recebemos os produtos antes do prazo da entrega. As ofertas cabe em nossos bolsos.",2018-03-20 12:47:53,2018-03-22 02:49:46,2018-04-02 00:00:00,2018-03-28 18:54:28,55.04,boleto,2018-03-28 02:49:46,SP,0
118288,moveis_decoracao,5.0,Adorei o produto ótimo e recomendo.,2017-10-06 19:10:16,2017-10-06 19:25:54,2017-10-30 00:00:00,2017-10-25 18:17:06,95.69,credit_card,2017-10-13 19:25:54,SP,0
118302,informatica_acessorios,5.0,Tudo ok,2018-03-17 12:11:45,2018-03-17 12:29:22,2018-04-19 00:00:00,2018-04-07 10:07:48,456.28,credit_card,2018-03-27 12:29:22,SP,0
118306,instrumentos_musicais,1.0,"A compra foi feita Há mais de um mes e ainda nao recebi o produto, sendo que a necessidade dele para o projeto já passou",2017-04-12 16:35:43,2017-04-12 16:45:12,2017-05-15 00:00:00,2017-05-17 11:38:22,15.86,credit_card,2017-04-28 16:45:12,SP,0


In [138]:
pd.options.display.max_colwidth = 300
msg_df = text_copy[['message']]
msg_df

,message
0,"Perfeito, produto entregue antes do combinado."
2,Chegou antes do prazo previsto e o produto surpreendeu pela qualidade. Muito satisfatório.
4,Gostei pois veio no prazo determinado .
8,"Na descrição do produto quando fui efetuar a compra, estava detalhado os pentes de disfarce.\r\nQuando recebe o produto verifiquei e notei que não havia os pentes 0.5 e 1.5"
10,"A caixa do produto veio com uma pequena avariação, mas de resto veio perfeito..."
...,...
118282,"Amo comprar no baratheon, sempre recebemos os produtos antes do prazo da entrega. As ofertas cabe em nossos bolsos."
118288,Adorei o produto ótimo e recomendo.
118302,Tudo ok
118306,"A compra foi feita Há mais de um mes e ainda nao recebi o produto, sendo que a necessidade dele para o projeto já passou"


In [139]:
text_copy['message'] = \
text_copy['message'].map(lambda x: re.sub('[,\.!?></]', '', x))
text_copy['message'] = \
text_copy['message'].map(lambda x: x.lower())

In [140]:
text_copy[['message']]

,message
0,perfeito produto entregue antes do combinado
2,chegou antes do prazo previsto e o produto surpreendeu pela qualidade muito satisfatório
4,gostei pois veio no prazo determinado
8,na descrição do produto quando fui efetuar a compra estava detalhado os pentes de disfarce\r\nquando recebe o produto verifiquei e notei que não havia os pentes 05 e 15
10,a caixa do produto veio com uma pequena avariação mas de resto veio perfeito
...,...
118282,amo comprar no baratheon sempre recebemos os produtos antes do prazo da entrega as ofertas cabe em nossos bolsos
118288,adorei o produto ótimo e recomendo
118302,tudo ok
118306,a compra foi feita há mais de um mes e ainda nao recebi o produto sendo que a necessidade dele para o projeto já passou


In [141]:
text_copy['message'] = text_copy['message'].replace('0','',regex=True).astype(str)
text_copy['message'] = text_copy['message'].replace('1','',regex=True).astype(str)
text_copy['message'] = text_copy['message'].replace('2','',regex=True).astype(str)
text_copy['message'] = text_copy['message'].replace('3','',regex=True).astype(str)
text_copy['message'] = text_copy['message'].replace('4','',regex=True).astype(str)
text_copy['message'] = text_copy['message'].replace('5','',regex=True).astype(str)
text_copy['message'] = text_copy['message'].replace('6','',regex=True).astype(str)
text_copy['message'] = text_copy['message'].replace('7','',regex=True).astype(str)
text_copy['message'] = text_copy['message'].replace('8','',regex=True).astype(str)
text_copy['message'] = text_copy['message'].replace('9','',regex=True).astype(str)
text_copy['message'] = text_copy['message'].replace('borrower added on ','',regex=True).astype(str)

In [143]:
def identify_tokens(row):
    message = row['message']
    tokens = nltk.word_tokenize(message)
    token_words = [w for w in tokens if w.isalpha()]
    return token_words


In [144]:
text_copy['message_clear'] = text_copy.apply(identify_tokens, axis=1)

In [145]:
text_copy[['message','message_clear']]

,message,message_clear
0,perfeito produto entregue antes do combinado,"[perfeito, produto, entregue, antes, do, combinado]"
2,chegou antes do prazo previsto e o produto surpreendeu pela qualidade muito satisfatório,"[chegou, antes, do, prazo, previsto, e, o, produto, surpreendeu, pela, qualidade, muito, satisfatório]"
4,gostei pois veio no prazo determinado,"[gostei, pois, veio, no, prazo, determinado]"
8,na descrição do produto quando fui efetuar a compra estava detalhado os pentes de disfarce\r\nquando recebe o produto verifiquei e notei que não havia os pentes e,"[na, descrição, do, produto, quando, fui, efetuar, a, compra, estava, detalhado, os, pentes, de, disfarce, quando, recebe, o, produto, verifiquei, e, notei, que, não, havia, os, pentes, e]"
10,a caixa do produto veio com uma pequena avariação mas de resto veio perfeito,"[a, caixa, do, produto, veio, com, uma, pequena, avariação, mas, de, resto, veio, perfeito]"
...,...,...
118282,amo comprar no baratheon sempre recebemos os produtos antes do prazo da entrega as ofertas cabe em nossos bolsos,"[amo, comprar, no, baratheon, sempre, recebemos, os, produtos, antes, do, prazo, da, entrega, as, ofertas, cabe, em, nossos, bolsos]"
118288,adorei o produto ótimo e recomendo,"[adorei, o, produto, ótimo, e, recomendo]"
118302,tudo ok,"[tudo, ok]"
118306,a compra foi feita há mais de um mes e ainda nao recebi o produto sendo que a necessidade dele para o projeto já passou,"[a, compra, foi, feita, há, mais, de, um, mes, e, ainda, nao, recebi, o, produto, sendo, que, a, necessidade, dele, para, o, projeto, já, passou]"


In [146]:
stemming = PorterStemmer()

my_list = []

In [147]:
def stem_list(row):
    text_copy = row['message_clear']
    stemmed_list = [stemming.stem(message_clear) for message_clear in text_copy]
    return (stemmed_list)


text_copy['message_stemmed'] = text_copy.apply(stem_list, axis=1)

In [148]:
text_copy[['message','message_clear','message_stemmed']]

,message,message_clear,message_stemmed
0,perfeito produto entregue antes do combinado,"[perfeito, produto, entregue, antes, do, combinado]","[perfeito, produto, entregu, ant, do, combinado]"
2,chegou antes do prazo previsto e o produto surpreendeu pela qualidade muito satisfatório,"[chegou, antes, do, prazo, previsto, e, o, produto, surpreendeu, pela, qualidade, muito, satisfatório]","[chegou, ant, do, prazo, previsto, e, o, produto, surpreendeu, pela, qualidad, muito, satisfatório]"
4,gostei pois veio no prazo determinado,"[gostei, pois, veio, no, prazo, determinado]","[gostei, poi, veio, no, prazo, determinado]"
8,na descrição do produto quando fui efetuar a compra estava detalhado os pentes de disfarce\r\nquando recebe o produto verifiquei e notei que não havia os pentes e,"[na, descrição, do, produto, quando, fui, efetuar, a, compra, estava, detalhado, os, pentes, de, disfarce, quando, recebe, o, produto, verifiquei, e, notei, que, não, havia, os, pentes, e]","[na, descrição, do, produto, quando, fui, efetuar, a, compra, estava, detalhado, os, pent, de, disfarc, quando, receb, o, produto, verifiquei, e, notei, que, não, havia, os, pent, e]"
10,a caixa do produto veio com uma pequena avariação mas de resto veio perfeito,"[a, caixa, do, produto, veio, com, uma, pequena, avariação, mas, de, resto, veio, perfeito]","[a, caixa, do, produto, veio, com, uma, pequena, avariação, ma, de, resto, veio, perfeito]"
...,...,...,...
118282,amo comprar no baratheon sempre recebemos os produtos antes do prazo da entrega as ofertas cabe em nossos bolsos,"[amo, comprar, no, baratheon, sempre, recebemos, os, produtos, antes, do, prazo, da, entrega, as, ofertas, cabe, em, nossos, bolsos]","[amo, comprar, no, baratheon, sempr, recebemo, os, produto, ant, do, prazo, da, entrega, as, oferta, cabe, em, nosso, bolso]"
118288,adorei o produto ótimo e recomendo,"[adorei, o, produto, ótimo, e, recomendo]","[adorei, o, produto, ótimo, e, recomendo]"
118302,tudo ok,"[tudo, ok]","[tudo, ok]"
118306,a compra foi feita há mais de um mes e ainda nao recebi o produto sendo que a necessidade dele para o projeto já passou,"[a, compra, foi, feita, há, mais, de, um, mes, e, ainda, nao, recebi, o, produto, sendo, que, a, necessidade, dele, para, o, projeto, já, passou]","[a, compra, foi, feita, há, mai, de, um, me, e, ainda, nao, recebi, o, produto, sendo, que, a, necessidad, dele, para, o, projeto, já, passou]"


In [149]:
stops = stopwords.words('portuguese')
def remove_stops(row):
    my_list = row['message_stemmed']
    meaningful_message = [w for w in my_list if not w in stops]
    return (meaningful_message)

text_copy['stem_meaningful'] = text_copy.apply(remove_stops, axis=1)

In [150]:
text_copy[['message','message_clear','message_stemmed','stem_meaningful']]

,message,message_clear,message_stemmed,stem_meaningful
0,perfeito produto entregue antes do combinado,"[perfeito, produto, entregue, antes, do, combinado]","[perfeito, produto, entregu, ant, do, combinado]","[perfeito, produto, entregu, combinado]"
2,chegou antes do prazo previsto e o produto surpreendeu pela qualidade muito satisfatório,"[chegou, antes, do, prazo, previsto, e, o, produto, surpreendeu, pela, qualidade, muito, satisfatório]","[chegou, ant, do, prazo, previsto, e, o, produto, surpreendeu, pela, qualidad, muito, satisfatório]","[chegou, prazo, previsto, produto, surpreendeu, qualidad, satisfatório]"
4,gostei pois veio no prazo determinado,"[gostei, pois, veio, no, prazo, determinado]","[gostei, poi, veio, no, prazo, determinado]","[gostei, veio, prazo, determinado]"
8,na descrição do produto quando fui efetuar a compra estava detalhado os pentes de disfarce\r\nquando recebe o produto verifiquei e notei que não havia os pentes e,"[na, descrição, do, produto, quando, fui, efetuar, a, compra, estava, detalhado, os, pentes, de, disfarce, quando, recebe, o, produto, verifiquei, e, notei, que, não, havia, os, pentes, e]","[na, descrição, do, produto, quando, fui, efetuar, a, compra, estava, detalhado, os, pent, de, disfarc, quando, receb, o, produto, verifiquei, e, notei, que, não, havia, os, pent, e]","[descrição, produto, efetuar, compra, detalhado, pent, disfarc, receb, produto, verifiquei, notei, havia, pent]"
10,a caixa do produto veio com uma pequena avariação mas de resto veio perfeito,"[a, caixa, do, produto, veio, com, uma, pequena, avariação, mas, de, resto, veio, perfeito]","[a, caixa, do, produto, veio, com, uma, pequena, avariação, ma, de, resto, veio, perfeito]","[caixa, produto, veio, pequena, avariação, ma, resto, veio, perfeito]"
...,...,...,...,...
118282,amo comprar no baratheon sempre recebemos os produtos antes do prazo da entrega as ofertas cabe em nossos bolsos,"[amo, comprar, no, baratheon, sempre, recebemos, os, produtos, antes, do, prazo, da, entrega, as, ofertas, cabe, em, nossos, bolsos]","[amo, comprar, no, baratheon, sempr, recebemo, os, produto, ant, do, prazo, da, entrega, as, oferta, cabe, em, nosso, bolso]","[amo, comprar, baratheon, sempr, recebemo, produto, prazo, entrega, oferta, cabe, bolso]"
118288,adorei o produto ótimo e recomendo,"[adorei, o, produto, ótimo, e, recomendo]","[adorei, o, produto, ótimo, e, recomendo]","[adorei, produto, ótimo, recomendo]"
118302,tudo ok,"[tudo, ok]","[tudo, ok]","[tudo, ok]"
118306,a compra foi feita há mais de um mes e ainda nao recebi o produto sendo que a necessidade dele para o projeto já passou,"[a, compra, foi, feita, há, mais, de, um, mes, e, ainda, nao, recebi, o, produto, sendo, que, a, necessidade, dele, para, o, projeto, já, passou]","[a, compra, foi, feita, há, mai, de, um, me, e, ainda, nao, recebi, o, produto, sendo, que, a, necessidad, dele, para, o, projeto, já, passou]","[compra, feita, mai, ainda, nao, recebi, produto, sendo, necessidad, projeto, passou]"


In [151]:
def rejoin_words(row):
    text_copy = row['stem_meaningful']
    joined_message= ( " ".join(text_copy))
    return joined_message

text_copy['processed'] = text_copy.apply(rejoin_words, axis=1)
text_copy

data_lemmatized = text_copy.copy()

In [152]:
cols_to_drop = ['message', 'message_clear', 'message_stemmed', 'stem_meaningful']
text_copy.drop(cols_to_drop, inplace=True, axis=1)
text_copy ['processed']

0                                                                        perfeito produto entregu combinado
2                                           chegou prazo previsto produto surpreendeu qualidad satisfatório
4                                                                             gostei veio prazo determinado
8         descrição produto efetuar compra detalhado pent disfarc receb produto verifiquei notei havia pent
10                                              caixa produto veio pequena avariação ma resto veio perfeito
                                                        ...                                                
118282                         amo comprar baratheon sempr recebemo produto prazo entrega oferta cabe bolso
118288                                                                       adorei produto ótimo recomendo
118302                                                                                              tudo ok
118306                      